# Visualization for Weather Forecasting

ClimateLearn provides tools to generate visualizations of model predictions for both forecasting and [downscaling](https://uaf-snap.org/how-do-we-do-it/downscaling). In this tutorial, we'll see how to visualize bias and mean bias. This tutorial is intended for use in Google Colab. Before starting, ensure that you are on a GPU runtime.

## Google Colab setup

You might need to restart the kernel after installing ClimateLearn so that your Colab environment knows to use the correct package versions.

In [ ]:
!pip install climate-learn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## Data preparation
The second cell of this section can be skipped if the data is already downloaded to your Drive. See the "Data Processing" notebook for mor details.

In [ ]:
root = "/content/drive/MyDrive/ClimateLearn"
source = "weatherbench"
dataset = "era5"
resolution = "5.625"
variable = "2m_temperature"
years = range(1979, 2018)

In [ ]:
from climate_learn.data import download
download(root=root, source=source, dataset=dataset, resolution=resolution, variable=variable)

In [ ]:
from climate_learn.data import DataModuleArgs, DataModule
from climate_learn.data.climate_dataset.args import ERA5Args
from climate_learn.data.tasks.args import ForecastingArgs

data_args = ERA5Args(
    root_dir=f"{root}/data/{source}/{dataset}/{resolution}/",
    variables=[variable],
    years=years
)

forecasting_args = ForecastingArgs(
    dataset_args=data_args,
    in_vars=[variable],
    out_vars=[variable],
    pred_range=3*24,
    subsample=6
)

data_module_args = DataModuleArgs(
    task_args=forecasting_args,
    train_start_year=1979,
    val_start_year=2015,
    test_start_year=2017,
    end_year=2018
)

data_module = DataModule(
    data_module_args=data_module_args,
    batch_size=128,
    num_workers=1
)

## Model training

See the "Model Training & Evaluation" notebook for more details.

In [ ]:
from climate_learn.models import load_model
from torch.optim import AdamW

model_kwargs = {
    "in_channels": 1,
    "out_channels": 1,
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
    "optimizer": AdamW
}

model_module = load_model(
    name="resnet",
    task="forecasting",
    model_kwargs=model_kwargs,
    optim_kwargs=optim_kwargs
)

In [ ]:
from climate_learn.models import set_climatology
set_climatology(model_module, data_module)

In [ ]:
from climate_learn.training import Trainer

trainer = Trainer(
    seed=0,
    accelerator="gpu",
    precision=16,
    max_epochs=1
)

trainer.fit(model_module, data_module)

## Visualization

We visualize the **bias**, given by the difference in the predicted and the ground truth values.

Visualization is done on the test set. ClimateLearn allows you to specify the exact times to visualize. Alternatively, you can specify a number $n$, and ClimateLearn will randomly sample $n$ times from the test set.

In [ ]:
from climate_learn.utils import visualize
visualize(model_module, data_module, samples=["2017-06-01:12", "2017-08-01:18"])

In addition to visualizing the bias the model has for each individual data point, we can also visualize the mean bias across the entire test set.

In [ ]:
from climate_learn.utils import visualize_mean_bias
visualize_mean_bias(model_module.cuda(), data_module)